In [9]:
import requests
import xmltodict
import os
import json
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 API 키 불러오기
load_dotenv()
kipris_key = os.getenv("KIPRIS_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=openai_api_key)

# 사용자 질문에서 키워드 추출
def extract_keywords_from_question(question):
  prompt = f"""
너는 특허 검색을 도와주는 AI야. 사용자가 질문한 문장에서 중요도에 따라 키워드를 구분해서 추출해줘.

조건:
- 중요 키워드(핵심)와 보조 키워드(있으면 좋음)를 구분해줘.
- JSON 형식으로 출력해줘. (주요 키워드: [], 보조 키워드: [])

질문: "{question}"
"""

  completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": "당신은 특허 검색 키워드 추출 전문가입니다."},
        {"role": "user", "content": prompt}
      ]
  )

  keywords_json = completion.choices[0].message.content.strip()

  try:
    keywords_dict = json.loads(keywords_json)
  except json.JSONDecodeError:
    keywords = [kw.strip() for kw in keywords_json.split(",")]
    keywords_dict = {"주요 키워드": keywords, "보조 키워드": []}

  return keywords_dict

# 특허 검색
def search_all_patents(keyword):
  all_patents = []
  page_no = 1

  while True:
    url = (
      f"http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getWordSearch"
      f"?word={keyword}&year=0&patent=Y&utility=Y"
      f"&numOfRows=100&pageNo={page_no}&ServiceKey={kipris_key}"
    )
    response = requests.get(url)

    if response.status_code == 200:
      xml_data = xmltodict.parse(response.content)
      body = xml_data['response']['body']
      items = body['items'].get('item')

      if not items:
        break

      if isinstance(items, dict):
        items = [items]

      all_patents.extend(items)

      if 'totalCount' in body:
        total_count = int(body['totalCount'])
        if len(all_patents) >= total_count:
          break
      else:
        break

      page_no += 1
    else:
      print("❌ API 요청 실패:", response.status_code)
      break

  return all_patents

# 특허 포맷팅
def format_patents(patent_items):
  formatted = []
  for idx, item in enumerate(patent_items, start=1):
    application_number = item['applicationNumber']
    title = item['inventionTitle']
    register_status = item['registerStatus']
    abstract = item.get('astrtCont', '내용 없음')

    formatted.append(
        f"📌 번호: {idx}\n\n"
        f"📄 발명명: {title}\n"
        f"📌 출원번호: {application_number}\n\n"
        f"📌 특허내용: {abstract}\n\n"
        f"📌 특허등록상태: {register_status}\n"
        f"🔗 상세보기: https://kpat.kipris.or.kr/kpat/biblioa.do?method=biblio&applicationNumber={application_number}\n"
    )
  return "\n".join(formatted)

# 문장 임베딩
def get_embedding(text):
  response = client.embeddings.create(
      model="text-embedding-ada-002",
      input=text
  )
  return np.array(response.data[0].embedding)

# 코사인 유사도 계산
def cosine_similarity(vec1, vec2):
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 사용자 질문과 유사 특허 찾기
def find_similar_patents(user_question, patents, top_n=5):
  user_emb = get_embedding(user_question)
  similarities = []

  for item in patents:
    title = item['inventionTitle']
    abstract = item.get('astrtCont', '내용 없음')
    text = title + " " + abstract
    patent_emb = get_embedding(text)

    similarity = cosine_similarity(user_emb, patent_emb)
    similarities.append((item, similarity))

  similarities.sort(key=lambda x: x[1], reverse=True)
  return similarities[:top_n]

# 유사 특허 포맷팅
def format_similar_patents(similarities):
  formatted = []
  for idx, (item, sim_score) in enumerate(similarities, start=1):
    application_number = item['applicationNumber']
    title = item['inventionTitle']
    register_status = item['registerStatus']
    url = f"https://kpat.kipris.or.kr/kpat/biblioa.do?method=biblio&applicationNumber={application_number}"

    formatted.append(
        f"📌 번호: {idx}\n"
        f"📄 발명명: {title}\n"
        f"📌 출원번호: {application_number}\n"
        f"📋 유사도: {sim_score * 100:.2f}%\n"
        f"📌 특허등록상태: {register_status}\n"
        f"🔗 상세보기: {url}\n"
    )
  return "\n".join(formatted)

# 등록 가능성 및 차별화 포인트 분석
def assess_patentability_with_score(user_question, patents):
  prompt = f"""
당신은 특허 등록 가능성을 판별하는 AI입니다.

사용자 질문: "{user_question}"

다음은 검색된 유사 특허 목록입니다:

"""

  for idx, item in enumerate(patents, start=1):
    title = item['inventionTitle']
    abstract = item.get('astrtCont', '내용 없음')
    prompt += f"{idx}. 제목: {title}\n내용: {abstract}\n\n"

  prompt += """
요구사항:
- 등록 가능성(%)를 수치로 제시하세요.
- 차별화 포인트를 1~2개 추천하세요. (예: LLM 최적화, 새로운 데이터셋 활용 등)

출력 포맷 예시:
"등록 가능성: 75%
차별화 포인트: '특정 알고리즘 최적화', '응용 분야 확대'"
"""

  completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": "당신은 특허 심사관입니다."},
        {"role": "user", "content": prompt}
      ]
  )

  return completion.choices[0].message.content.strip()

# 메인 실행
if __name__ == "__main__":
  user_question = input("📝 사용자 질문을 입력하세요: ")

  # 1. 키워드 추출
  keywords_info = extract_keywords_from_question(user_question)
  print(f"🔎 추출된 키워드: {keywords_info}")

  all_patents = []

  # 2. 주요 키워드 검색
  for keyword in keywords_info.get("주요 키워드", []):
    patents = search_all_patents(keyword)
    if patents:
      all_patents.extend(patents)

  # 3. 보조 키워드 검색
  for keyword in keywords_info.get("보조 키워드", []):
    patents = search_all_patents(keyword)
    if patents:
      all_patents.extend(patents)

  # 4. 중복 제거
  seen = set()
  unique_patents = []
  for patent in all_patents:
    app_no = patent['applicationNumber']
    if app_no not in seen:
      seen.add(app_no)
      unique_patents.append(patent)

  # 5. 결과 출력
  if unique_patents:
    print("\n🤖 검색된 특허 목록:\n")
    formatted_output = format_patents(unique_patents)
    print(formatted_output)

    # 6. 유사 특허 찾기
    print("\n🧠 사용자 질문과 유사한 특허 Top 5:\n")
    similarities = find_similar_patents(user_question, unique_patents)
    similar_patents_output = format_similar_patents(similarities)
    print(similar_patents_output)

    # 7. 등록 가능성 평가 + 차별화 포인트 제시
    print("\n📈 등록 가능성 및 차별화 포인트 분석:\n")
    assessment = assess_patentability_with_score(user_question, [item for item, _ in similarities])
    print(assessment)

  else:
    print("❌ 관련 특허를 찾지 못했습니다.")


🔎 추출된 키워드: {'주요 키워드': ['개인 맞춤형', '학습 경로 추천', '교육 플랫폼'], '보조 키워드': []}

🤖 검색된 특허 목록:

📌 번호: 1

📄 발명명: 개인맞춤형 약물치료관리 방법 및 이를 위한 시스템
📌 출원번호: 1020220182493

📌 특허내용: 본 발명은 개인맞춤형 약물치료관리 방법 및 이를 위한 시스템에 관한 것이다. 구체적으로 본 발명은 사용자의 의약품 및 약물치료에 대한 현황을 관리해 주는 주체로 약국 또는 약국서버를 활용함으로써 사용자의 개인맞춤형 약물치료관리 서비스를 제공하는 것에 관한 것이다.

📌 특허등록상태: 공개
🔗 상세보기: https://kpat.kipris.or.kr/kpat/biblioa.do?method=biblio&applicationNumber=1020220182493

📌 번호: 2

📄 발명명: 개인맞춤형 뇌 자극 시스템 및 이를 이용한 뇌 자극 방법
📌 출원번호: 1020230194662

📌 특허내용: 본 발명의 일 실시예에 따른 개인맞춤형 뇌 자극 시스템은, 단말; 및 상기 단말과 통신하는 뇌 자극 디바이스;를 포함하는 시스템에 있어서, 상기 단말에 설치된 애플리케이션을 통해, 사용자와 관련된 우울증의 증상과 정도를 평가하기 위한 벡 우울척도(Beck Depression Inventory, BDI) 정보 및 상기 우울증에 대한 민감성을 측정하는 라이덴 우울 감수성 지수(Leiden Index of Depression Sensitivity-Revised, LEIDS-R) 정보 중 적어도 하나인 제1 정보가 수집되고, 상기 단말은 상기 수집된 제1 정보를 상기 뇌 자극 디바이스로 전송하며, 상기 뇌 자극 디바이스는, 상기 수신한 제1 정보를 기초로, 우울증 관련 기설정된 복수의 상태 중 상기 사용자가 제1 상태에 해당하는 것으로 결정하는 제어부; 및 상기 사용자의 뇌의 복수의 영역에서 동기화된 진동을 동조시키기 위해, 상기 결정된 제1 상태에 따라 달라지는 자극을 상기 사용자의 뇌에 전

KeyboardInterrupt: 